<a href="https://colab.research.google.com/github/bhuvanesh-o/personal-portfolio/blob/main/Copy_of_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
#1

# we should read the files and then load them into pandas for preprocessing and model training

# Load the TSV data into pandas DataFrames

train_data = pd.read_csv(train_file_path, sep="\t", names=["label", "message"])
test_data = pd.read_csv(test_file_path, sep="\t", names=["label", "message"])


In [ ]:
#2

# encode them into "ham" and "spam" (0 and 1) - becuase in the tsv file we have ham and spam

# Convert labels to 0 and 1
label_map = {'ham': 0, 'spam': 1}
train_labels = [label_map[label] for label in train_data['label']]
test_labels = [label_map[label] for label in test_data['label']]

In [ ]:
#3

# tokenise and padding

# Tokenize and pad sequences

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000, oov_token="<OOV>") # Tokenizer is a Keras utility that turns words -> integers & takes top 10000 words (in this)
tokenizer.fit_on_texts(train_data['message']) # it will assign a number for every word in the message in a dictionary format

train_sequences = tokenizer.texts_to_sequences(train_data['message']) # we will be converting the messages into a sequence of numbers for BOTH test and train -----> "Free call now" → [2, 3, 4]
test_sequences = tokenizer.texts_to_sequences(test_data['message'])



max_length = 100

train_padded = tf.keras.preprocessing.sequence.pad_sequences(
    train_sequences, maxlen=max_length, padding='post', truncating='post'   # we are padding in order to have same length and here it is 100
)

test_padded = tf.keras.preprocessing.sequence.pad_sequences(
    test_sequences, maxlen=max_length, padding='post', truncating='post'
)


In [ ]:
#4

# Building the Model

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=16, input_length=100), # top 10000 words, each word will be represented as a 16-dimensional vector, padded/truncated to 100 words
    tf.keras.layers.GlobalAveragePooling1D(),  # Output shape becomes just one 16-dimensional vector per message.
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [ ]:
#5

# Convert labels to NumPy array
train_labels = np.array(train_labels)

# Train & Compile the Model

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_padded, train_labels, epochs=25, validation_split=0.2)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

   # Convert text to sequence of word indices
    seq = tokenizer.texts_to_sequences([pred_text])

    # Pad sequence to match training input length
    padded = tf.keras.preprocessing.sequence.pad_sequences(
        seq, maxlen=max_length, padding='post', truncating='post'
    )

    # Predict the probability that the message is spam
    prob = model.predict(padded)[0][0]

    # Determine the label based on probability
    label = 'spam' if prob >= 0.5 else 'ham'

    # Return as a list: [probability, label]
    return [float(prob), label]


pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
